In [18]:
# %load /Users/hwayment/ipynb_defaults.py
%load_ext autoreload
%autoreload 2

%pylab inline
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


## Code to reproduce DegScore from Kaggle datasets

In [3]:
def encode_input(df, window_size=1, pad=10, seq=True, struct=True):
    '''Creat input/output for regression model for predicting structure probing data.
    Inputs:
    
    dataframe (in EternaBench RDAT format)
    window_size: size of window (in one direction). so window_size=1 is a total window size of 3
    pad: number of nucleotides at start to not include
    seq (bool): include sequence encoding
    struct (bool): include bpRNA structure encoding
    
    Outputs:
    Input array (n_samples x n_features): array of windowed input features
    feature_names (list): feature names
    
    '''
    MAX_LEN = 68
    
    inpts = []
    labels = []

    feature_kernel=[]
    if seq:
        feature_kernel.extend(['A','U','G','C'])
    if struct:
        feature_kernel.extend(['H','E','I','M','B','S'])

    feature_names = ['%s_%d' % (k, val) for val in range(-1*window_size, window_size+1) for k in feature_kernel]
    
    for i, row in df.iterrows():
        
        arr = np.zeros([MAX_LEN,len(feature_kernel)])
        
        for index in range(pad,MAX_LEN):
            ctr=0

            #encode sequence
            if seq:
                
                for char in ['A','U','G','C']:
                    if row['sequence'][index]==char:
                        arr[index,ctr]+=1
                    ctr+=1

            if struct:
                for char in ['H','E','I','M','B','S']:
                    if row['predicted_loop_type'][index]==char:
                        arr[index,ctr]+=1
                    ctr+=1

        # add zero padding to the side

        padded_arr = np.vstack([np.zeros([window_size,len(feature_kernel)]),arr[pad:], np.zeros([window_size,len(feature_kernel)])])

        for index in range(pad,MAX_LEN):
            new_index = index+window_size-pad
            tmp = padded_arr[new_index-window_size:new_index+window_size+1]
            inpts.append(tmp.flatten())
            labels.append('%s_%d' % (row['id'], index))
            
    return np.array(inpts), feature_names, labels

def encode_output(df, data_type='reactivity', pad=10):
    '''Creat input/output for regression model for predicting structure probing data.
    Inputs:
    
    dataframe (in EternaBench RDAT format)
    data_type: column name for degradation
    window_size: size of window (in one direction). so window_size=1 is a total window size of 3
    pad: number of nucleotides at start to not include
    seq (bool): include sequence encoding
    struct (bool): include bpRNA structure encoding
    
    Outputs:
    output array (n_samples): array of reactivity values
    
    '''
    MAX_LEN = 68
    
    outpts = []
    labels = []
    # output identity should be in form id_00073f8be_0

    for i, row in df.iterrows():
        for index in range(pad,MAX_LEN):
            outpts.append(row[data_type][index])
            labels.append('%s_%d' % (row['id'], index))
    return outpts, labels

In [7]:
kaggle_train = pd.read_json('train.json',lines=True)
kaggle_train = kaggle_train.loc[kaggle_train['SN_filter']==1]

kaggle_test = pd.read_json('test.json',lines=True)

inputs_train, feature_names, _ = encode_input(kaggle_train, window_size=12)
inputs_test, _, test_labels = encode_input(kaggle_test, window_size=12)

reg_models = []

for output_type in ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C','deg_50C']:
    outputs_train, outputs_labels = encode_output(kaggle_train, data_type=output_type)
    
    # Clip negative values to 0
    outputs_train = np.clip(outputs_train, 0, 100)

    reg = Ridge(alpha=0.15)
    print('Fitting %s ...' % output_type)
    reg.fit(inputs_train, outputs_train)
    reg_models.append(reg)
    test_prediction = reg.predict(inputs_test)

Fitting reactivity ...
Fitting deg_Mg_pH10 ...
Fitting deg_pH10 ...
Fitting deg_Mg_50C ...
Fitting deg_50C ...


## Using the existing class that contains "DegScore-2.1"

In [30]:
sys.path.append('/Users/hwayment/das/github/arnie-deg')
from DegScore import DegScore

In [31]:
sequence = 'GGGGAAAACCCC'
mdl = DegScore(sequence, structure='((((....))))')

print(mdl.degscore_by_position)
print(mdl.degscore)

[0.156 0.082 0.283 0.386 0.47  0.529 0.559 0.394 0.198 0.187 0.085 0.708]
4.037000000000002
